In [1]:
from IPython.display import display, HTML, Image
display(HTML("<style>.container { width:85% !important; }</style>"))
%config IPCompleter.use_jedi=False

In [34]:
import os
from datasets import load_dataset, Dataset
from evaluate import load
from tqdm.notebook import tqdm
import pickle
import pandas as pd

In [2]:
code_eval = load("code_eval")

In [3]:
os.environ["HF_ALLOW_CODE_EVAL"] = "1"

In [4]:
dset = load_dataset('cakiki/humaneval-codeparrot-small-eval_corrected', split='train')

Downloading:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

Using custom data configuration cakiki--humaneval-codeparrot-small-eval_corrected-00f5fee1fc9f5497


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /home/christopher/.cache/huggingface/datasets/cakiki___parquet/cakiki--humaneval-codeparrot-small-eval_corrected-00f5fee1fc9f5497/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


In [5]:
df = dset.to_pandas()

In [6]:
df

,index,task_id,prompt,canonical_solution,test,entry_point,cleaned_generation,full_generation,temperature,top_k,...,do_sample,num_beams,num_beam_groups,num_return_sequences,num_tokens_to_generate,codebert-bert-score-prec,codebert-bert-score-recall,codebert-bert-score-f1,edit_distance,reference
0,0,HumanEval/2,\n\ndef truncate_number(number: float) -> floa...,return number % 1.0\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",truncate_number,[\n\ndef truncate_number(number: float) -> flo...,[\n\ndef truncate_number(number: float) -> flo...,1.4,50,...,True,1,1,200,128,"[0.760907769203186, 0.7050846815109253, 0.7610...","[0.8042548894882202, 0.8064022064208984, 0.802...","[0.7819811105728149, 0.7523476481437683, 0.781...","[0.008456659619450317, 0.006472491909385114, 0...","\n\n\nMETADATA = {\n 'author': 'jt',\n '..."
1,1,HumanEval/7,from typing import List\n\n\ndef filter_by_sub...,return [x for x in strings if substring in...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",filter_by_substring,[from typing import List\n\n\ndef filter_by_su...,[from typing import List\n\n\ndef filter_by_su...,1.4,50,...,True,1,1,200,128,"[0.8089243173599243, 0.7974163293838501, 0.781...","[0.7688605189323425, 0.7854522466659546, 0.805...","[0.7883837819099426, 0.7913891077041626, 0.793...","[0.26666666666666666, 0.006622516556291391, 0....","\n\n\nMETADATA = {\n 'author': 'jt',\n '..."
2,2,HumanEval/15,"\n\ndef string_sequence(n: int) -> str:\n ""...",return ' '.join([str(x) for x in range(n +...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",string_sequence,[\n\ndef string_sequence(n: int) -> str:\n ...,[\n\ndef string_sequence(n: int) -> str:\n ...,1.4,50,...,True,1,1,200,128,"[0.7830531597137451, 0.7653099298477173, 0.778...","[0.7793910503387451, 0.7982255220413208, 0.779...","[0.7812178730964661, 0.7814212441444397, 0.778...","[0.12087912087912088, 0.08695652173913043, 0.2...","\n\n\nMETADATA = {\n 'author': 'jt',\n '..."
3,3,HumanEval/16,\n\ndef count_distinct_characters(string: str)...,return len(set(string.lower()))\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",count_distinct_characters,[\n\ndef count_distinct_characters(string: str...,[\n\ndef count_distinct_characters(string: str...,1.4,50,...,True,1,1,200,128,"[0.7902371287345886, 0.7685325741767883, 0.792...","[0.8768507242202759, 0.8553634881973267, 0.876...","[0.8312939405441284, 0.8096265196800232, 0.832...","[0.03854389721627409, 0.04395604395604396, 0.0...","\n\n\nMETADATA = {\n 'author': 'jt',\n '..."
4,4,HumanEval/22,"from typing import List, Any\n\n\ndef filter_i...","return [x for x in values if isinstance(x,...","\n\nMETADATA = {\n 'author': 'jt',\n 'da...",filter_integers,"[from typing import List, Any\n\n\ndef filter_...","[from typing import List, Any\n\n\ndef filter_...",1.4,50,...,True,1,1,200,128,"[0.7488330006599426, 0.8264870643615723, 0.815...","[0.8270582556724548, 0.8247451782226562, 0.784...","[0.7860041260719299, 0.8256152272224426, 0.799...","[0.04173913043478261, 0.37681159420289856, 0.2...","\n\n\nMETADATA = {\n 'author': 'jt',\n '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3399,2696,HumanEval/138,"\ndef is_equal_to_sum_even(n):\n """"""Evaluat...",return n%2 == 0 and n >= 8\n,def check(candidate):\n assert candidate(4)...,is_equal_to_sum_even,"[\ndef is_equal_to_sum_even(n):\n """"""Evalua...","[\ndef is_equal_to_sum_even(n):\n """"""Evalua...",1.0,50,...,False,1,1,1,128,[0.0],[0.0],[0.0],[0.0],\ndef check(candidate):\n assert candidate(...
3400,2697,HumanEval/151,\ndef double_the_difference(lst):\n '''\n ...,return sum([i**2 for i in lst if i > 0 and...,def check(candidate):\n\n # Check some simp...,double_the_difference,[\ndef double_the_difference(lst):\n '''\n ...,[\ndef double_the_difference(lst):\n '''\n ...,1.0,50,...,False,1,1,1,128,[0.7692716717720032],[0.8172289133071899],[0.7925254106521606],[0.008948545861297539],\ndef check(candidate):\n\n # Check some si...
3401,2698,HumanEval/152,"\ndef compare(ga

In [7]:
c = (df.itertuples())

In [8]:
a = next(c)

In [11]:
a.index

0

In [9]:
%%time
pass_at_k, meta = code_eval.compute(
            references=["\n" + a.test + "\n" + a.entry_point], predictions=[a.cleaned_generation.tolist()], num_workers=28, k=[1,10,50,100,150,200]
        )

CPU times: user 2.15 s, sys: 11.8 s, total: 13.9 s
Wall time: 12.2 s


In [19]:
%%time
pass_at_k_list = []
meta_list = []

for t in tqdm(df.itertuples(), total=len(df)):
    pass_at_k, meta = code_eval.compute(
                references=["\n" + t.test + "\n" + t.entry_point], predictions=[t.cleaned_generation.tolist()], num_workers=28, k=[1,10,50,100,150,200]
            )
    pass_at_k_list.append(pass_at_k)
    meta_list.append(meta)
    with open(f'pass_at_k/passatk_{t.index}.pkl', 'wb') as f:
        pickle.dump(pass_at_k, f)

    with open(f'pass_at_k/meta_{t.index}.pkl', 'wb') as f:
        pickle.dump(meta, f)

  0%|          | 0/3404 [00:00<?, ?it/s]

Process Process-553752:
  File "/usr/lib/python3.7/shutil.py", line 494, in rmtree
    _rmtree_safe_fd(fd, path, onerror)
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/christopher/.cache/huggingface/modules/evaluate_modules/metric/code_eval/e98eda8cf8ea415a6855159b442b21ea24afe2fcadce6a37bf43e00aa439e2c0/execute.py", line 86, in unsafe_execute
    os.chdir = chdir
  File "/usr/lib/python3.7/contextlib.py", line 119, in __exit__
    next(self.gen)
  File "/home/christopher/.cache/huggingface/modules/evaluate_modules/metric/code_eval/e98eda8cf8ea415a6855159b442b21ea24afe2fcadce6a37bf43e00aa439e2c0/execute.py", line 115, in create_tempdir
    yield dirname
  File "/usr/lib/python3.7/tempfile.py", line 944, in __exit__
    self.cleanup()
  File "/usr/lib/python3.7/tempfile.py",

CPU times: user 1h 52min 42s, sys: 12h 29min 32s, total: 14h 22min 15s
Wall time: 12h 46min 1s


In [30]:
pass_at_k_df = pd.DataFrame(pass_at_k_list)

In [35]:
Dataset.from_pandas(pd.concat([df, pass_at_k_df], axis=1)).push_to_hub('cakiki/humaneval-codeparrot-small-eval_corrected')

Pushing dataset shards to the dataset hub:   0%|          | 0/2 [00:00<?, ?it/s]

/media/ssd/BIGSCIENCE/huggingface_hub/src/huggingface_hub/hf_api.py:2039: FutureWarning: `identical_ok` has no effect and is deprecated. It will be removed in 0.11.0.
  FutureWarning,


In [33]:
sum(pass_at_k_df['pass@10'].isna())

37

In [20]:
with open(f'passatk_list.pkl', 'wb') as f:
    pickle.dump(pass_at_k_list, f)

with open(f'meta_list.pkl', 'wb') as f:
    pickle.dump(meta_list, f)

In [17]:
# %%time
# dset = dset.map(lambda x: {'references':"\n" + x['test'] + "\n" + x['entry_point']})
# pass_at_k, meta = code_eval.compute(
#             references=dset['references'], predictions=dset['cleaned_generation'], num_workers=28, k=[1,10,50,100,150,200]
#         )

In [36]:
df.describe()

,index,temperature,top_k,top_p,typical_p,num_beams,num_beam_groups,num_return_sequences,num_tokens_to_generate
count,3404.000000,3404.000000,3404.0,3404.000000,3404.000000,3404.000000,3404.0,3404.00000,3404.0
mean,2260.521739,1.000000,50.0,0.755435,0.755435,2.076087,1.0,196.75000,128.0
std,1730.141033,0.279793,0.0,0.308394,0.308394,10.266723,0.0,22.99422,0.0
min,0.000000,0.600000,50.0,0.100000,0.100000,1.000000,1.0,1.00000,128.0
25%,850.750000,0.800000,50.0,0.500000,0.500000,1.000000,1.0,200.00000,128.0
50%,1701.500000,1.000000,50.0,1.000000,1.000000,1.000000,1.0,200.00000,128.0
75%,4365.250000,1.200000,50.0,1.000000,1.000000,1.000000,1.0,200.00000,128.0
max,5549.000000,1.400000,50.0,1.000000,1.000000,100.000000,1.0,200.00000,128.0
